# Simple regular geometries

In [ ]:
import pycpt
import numpy as np

from time import clock
import fillit as pitfill

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

## Plotting functions

In [ ]:
def plotElevation( data, cmin, cmax, colormap):
    '''
    data: dataset to plot
    zmin,zmax: extent of the colormap
    colormap: to use    
    '''
    
    # Figure size is defined here
    fig = plt.figure(1, figsize=(6,6))
    
    ax = plt.gca()
    im = ax.imshow(data, interpolation='nearest', cmap=colormap,
                     vmin=cmin, vmax=cmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(im,cax=cax)
    plt.tight_layout()

    plt.show()
    plt.close(fig)

## Surface definition

In [ ]:
dx = 0.01
minX, maxX = 0.0, 10.0
minY, maxY = 0.0, 10.0

nx = int((maxX - minX)/dx)+1
ny = int((maxY - minY)/dx)+1

xcoords = np.linspace(minX, maxX, nx)
ycoords = np.linspace(minY, maxY, ny)

X, Y = np.meshgrid(xcoords, ycoords)

coords = np.vstack([X.ravel(), Y.ravel()])


height  = 1.0 - (np.sin(0.5*X*np.pi)**2 * np.sin(0.5*Y*np.pi)**2)
#height  += np.random.random(height.shape) * 0.001
height  += 0.25 * (X + 0.5*Y)

dem = height.reshape((ny,nx))

In [ ]:
diffcmap = pycpt.load.cmap_from_cptcity_url('cmocean/balance.cpt')
plotElevation( dem, dem.min(), dem.max(), diffcmap)

# Perform pit filling

We will now perform pit filling using one of the available priority-flood algorithms available in `fillit`.

For regular grids, the following `classes` are available:
+ `depressionFillingZhou`
+ `depressionFillingBarnes`

***

These 2 classes are based on the following papers:

Barnes, Lehman, Mulla. "Priority-Flood: An Optimal Depression-Filling and
Watershed-Labeling Algorithm for Digital Elevation Models". Computers & Geosciences.
Vol 62, Jan 2014, pp 117–127 - [link](https://www.sciencedirect.com/science/article/pii/S0098300413001337)

Zhou, Sun, Fu. "An efficient variant of the Priority-Flood algorithm for filling
depressions in raster digital elevation models". Computers & Geosciences.
Vol 90, Feb 2016, pp 87–96 - [link](https://www.sciencedirect.com/science/article/pii/S0098300416300553)

***

To call one of these classes, you will typically do as follows:

``` python
import fillit as pitfill

# Class initialisation
pitClass = pitfill.depressionFillingBarnes(dem)

# Performing pit filling
fillZ = pitClass.performPitFillingSruct()

```

Here we illustrate how this is done for the 2 classes...

### Barnes (2014)

In [ ]:
t0 = clock()
fillB = pitfill.depressionFillingBarnes(dem)
filledB = fillB.performPitFillingStruct()
print('\n+ Pit filling - Priority-flood Barnes Algorithm 2014 (%0.02f seconds)'% (clock() - t0))

### Zhou (2016)

It is worth mentionning that the performance of Zhou's algorithm in `fillit` performed a bit slower than the Barnes'one mainly because it also computes additional information such as pit labelling, pit volume evaluation, watershed definition and spill-over graph...

In [ ]:
t0 = clock()
fillZ = pitfill.depressionFillingZhou(dem,cartesian=True)
filledZ, LabelsZ, wLabelsZ, graph = fillZ.performPitFillingStruct(simple=False)
print('\n+ Pit filling - Priority-flood Zhou Algorithm 2014 (%0.02f seconds)'% (clock() - t0))

In [ ]:
print('+ Pit filling elevation difference between Zhou & Barnes: \n \t \t min: %0.02f - max: %0.02f \n' % ((filledB-filledZ).min(),(filledB-filledZ).max()))

We can visualise the filled elevation differences with the initial digital elevation model

In [ ]:
print('\t\t\t Zhou\'s pit filling')
topocmap = pycpt.load.cmap_from_cptcity_url('cmocean/amp.cpt')
plotElevation(filledZ-dem, 0, 0.4, topocmap)

# Perform pit filling with epsilon

Only implemented for the Barnes algorithm...

## Barnes' algo

In [ ]:
t0 = clock()
fillB2 = pitfill.depressionFillingBarnes(dem,eps=1.e-12,cartesian=True,first=0)
filledB2 = fillB2.performPitFillingStruct()
print('\n+ Pit filling+epsilon - Priority-flood Barnes Algorithm 2014 (%0.02f seconds)'% (clock() - t0))

In [ ]:
print('\t\t Barnes pit filling + eps')
plotElevation( filledB2, filledB2.min(), filledB2.max(), diffcmap)

print('\t\t Difference with or whithout eps')
plotElevation( filledB2-filledB, (filledB2-filledB).min(), (filledB2-filledB).max(), diffcmap)